Jose R F Junior
web2ajax@gmail.com

# Fine-tune Llama 2 in Google Colab



**Esta linha de código instala as seguintes bibliotecas Python:**

accelerate==0.21.0: Uma biblioteca que fornece um conjunto de ferramentas para acelerar o treinamento de modelos de linguagem em GPUs.

peft==0.4.0: Uma biblioteca que implementa a atenção LoRA, que é uma técnica de atenção que pode melhorar o desempenho de modelos de linguagem em tarefas de processamento de linguagem natural.

bitsandbytes==0.40.2: Uma biblioteca que fornece ferramentas para a quantização de modelos de linguagem, que é uma técnica que pode reduzir o tamanho e o consumo de memória de modelos de linguagem.

transformers==4.31.0: Uma biblioteca que fornece implementações de vários modelos de linguagem, incluindo o modelo Llama-2-7b-chat-hf, que é o modelo utilizado no código fornecido.

trl==0.4.7: Uma biblioteca que fornece ferramentas para o treinamento supervisionado de modelos de linguagem, que é o tipo de treinamento utilizado no código fornecido.

A opção -q indica que a instalação deve ser executada silenciosamente, sem exibir mensagens de progresso.

Após a instalação dessas bibliotecas, o código fornecido poderá ser executado corretamente.

In [5]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00


Esta linha de código importa as bibliotecas necessárias para o código fornecido. Aqui está uma explicação detalhada das bibliotecas importadas:

os: Esta biblioteca fornece funções para interagir com o sistema operacional, como criar diretórios e arquivos.
torch: Esta biblioteca é o framework de deep learning utilizado para treinar o modelo de linguagem.

datasets: Esta biblioteca fornece um conjunto de ferramentas para trabalhar com conjuntos de dados, como carregar, processar e visualizar dados.

transformers: Esta biblioteca fornece implementações de vários modelos de linguagem pré-treinados, incluindo o modelo Llama-2-7b-chat-hf.

AutoModelForCausalLM: Esta classe representa um modelo de linguagem pré-treinado para geração de linguagem.

AutoTokenizer: Esta classe representa um tokenizador pré-treinado para dividir texto em tokens.

BitsAndBytesConfig: Esta classe especifica as configurações para a quantização do modelo de linguagem.

HfArgumentParser: Esta classe é usada para parsear argumentos de linha de comando para o treinamento do modelo.

TrainingArguments: Esta classe define parâmetros para o treinamento do modelo, como o número de épocas, o tamanho do lote e a taxa de aprendizado.

pipeline: Esta classe fornece um mecanismo para treinar e usar pipelines de processamento de linguagem, como geração de linguagem.

logging: Esta biblioteca fornece funções para registrar mensagens de depuração e informações durante o treinamento do modelo.

peft: Esta biblioteca fornece implementações da atenção LoRA, que é uma técnica de atenção para modelos de linguagem.

LoraConfig: Esta classe especifica as configurações para a atenção LoRA no modelo de linguagem.

PeftModel: Esta classe representa um modelo de linguagem com atenção LoRA.
trl: Esta biblioteca fornece ferramentas para o treinamento supervisionado de modelos de linguagem.

SFTTrainer: Esta classe é usada para treinar um modelo de linguagem usando supervisão.

In [6]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer


**LQORA parâmetros:**

lora_r: Dimensão da atenção LoRA, que controla a quantidade de informação que é compartilhada entre diferentes partes do modelo.

lora_alpha: Parâmetro alpha para escalar a LoRA, que determina a força da atenção LoRA.

lora_dropout: Probabilidade de dropout para camadas LoRA, que reduz o overfitting durante o treinamento.

**bitsandbytes parâmetros:**

use_4bit: Ativa a carga do modelo base com precisão de 4 bits, o que reduz o tamanho do modelo e o consumo de memória.

bnb_4bit_compute_dtype: Tipo de dados de computação para modelos base com precisão de 4 bits.

bnb_4bit_quant_type: Tipo de quantização (fp4 ou nf4) para modelos base com precisão de 4 bits.

use_nested_quant: Ativa a quantização aninhada para modelos base com precisão de 4 bits (quantização dupla).

**TrainingArguments parâmetros:**

output_dir: Diretório de saída onde os resultados e os checkpoints do modelo serão armazenados.

num_train_epochs: Número de épocas de treinamento.

fp16: Habilita o treinamento em precisão de 16 bits (fp16) para reduzir o consumo de memória.

bf16: Habilita o treinamento em precisão de 16 bits bfloat16 (bf16) para reduzir ainda mais o consumo de memória, se disponível no hardware.

per_device_train_batch_size: Tamanho do lote de treinamento para cada GPU.

per_device_eval_batch_size: Tamanho do lote de avaliação para cada GPU.

gradient_accumulation_steps: Número de atualizações de etapa para acumular os gradientes.

gradient_checkpointing: Habilita o checkpointing de gradiente para melhorar o desempenho de treinamento em GPUs com pouca memória.

max_grad_norm: Máximo valor de norma de gradiente (clipping de gradiente) para evitar gradientes muito grandes que podem desestabilizar o treinamento.

learning_rate: Taxa de aprendizado inicial do otimizador AdamW.

weight_decay: Decaimento de peso a ser aplicado em todas as camadas, exceto em pesos de bias/LayerNorm.

optim: Otimizador a ser usado, neste caso, paged_adamw_32bit.

lr_scheduler_type: Tipo de agendador de taxa de aprendizado, neste caso, cosine.

max_steps: Número de etapas de treinamento (substitui num_train_epochs) para limitar o treinamento em etapas específicas.

warmup_ratio: Razão de etapas para um aquecimento linear (de 0 a taxa de aprendizado máxima) no início do treinamento.

group_by_length: Agrupa sequências em lotes com o mesmo comprimento, economizando memória e acelerando o treinamento consideravelmente.

save_steps: Intervalo de salvamento de checkpoints em etapas de atualização.
logging_steps: Intervalo de log de progresso em etapas de atualização.

**SFT parâmetros:**

max_seq_length: Máximo comprimento da sequência a ser usado.
packing: Empaqueta vários exemplos curtos na mesma sequência de entrada para aumentar a eficiência.

device_map: Mapa de dispositivos para carregar o modelo inteiro na GPU 0.

Em resumo, este código configura um treinamento personalizado para o modelo Llama-2-7b-chat-hf, otimizando-o para o uso eficiente de memória e desempenho de treinamento em GPUs, utilizando técnicas como quantização, gradient checkpointing e grupo de sequências.

In [7]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
# dataset_name = "mlabonne/guanaco-llama2-1k"
dataset_name = "JoseRFJunior/priming-llama2-1k"

# Fine-tuned model name
new_model = "llama-2-7b-priming"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs 1 para 15
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training 4 para 16
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [8]:
# Verificar chaves do conjunto de dados
print("Chaves do conjunto de dados:", dataset_name)

# Exibir algumas amostras do conjunto de dados
print("Algumas amostras do conjunto de dados:")
for i in range(5):  # Exibir as primeiras 5 amostras
    print(dataset_name[i])


Chaves do conjunto de dados: JoseRFJunior/priming-llama2-1k
Algumas amostras do conjunto de dados:
J
o
s
e
R


In [9]:
print(dataset_name)

JoseRFJunior/priming-llama2-1k



**Carregamento de dados:**

Load dataset (you can process it here): O código carrega o conjunto de dados de instruções especificado pelo dataset_name. Este conjunto de dados contém exemplos de instruções que o modelo será treinado para entender e processar.

**Carregamento de modelo e tokenizador:**

Load tokenizer and model with QLoRA configuration: O código carrega um tokenizador e um modelo utilizando a configuração QLoRA especificada nos parâmetros definidos anteriormente. O tokenizador é responsável por dividir as instruções em tokens individuais e o modelo é responsável por processar esses tokens e gerar respostas.

Compute_dtype = getattr(torch, bnb_4bit_compute_dtype): Esta linha de código define o tipo de dados de computação para o modelo, conforme especificado pelo parâmetro bnb_4bit_compute_dtype. Se o valor for float16, o modelo será treinado em precisão de 16 bits, enquanto se for bfloat16, o modelo será treinado em precisão de 16 bits bfloat16, que é mais eficiente em GPUs.

bnb_config = BitsAndBytesConfig(load_in_4bit=use_4bit, bnb_4bit_quant_type=bnb_4bit_quant_type, bnb_4bit_compute_dtype=compute_dtype, bnb_4bit_use_double_quant=use_nested_quant): Esta linha de código cria uma instância do objeto BitsAndBytesConfig, que contém configurações para a quantização do modelo. A quantização é uma técnica que reduz o tamanho do modelo e o consumo de memória, mas pode afetar o desempenho.

Check GPU compatibility with bfloat16: Esta linha de código verifica se a GPU disponível é compatível com bfloat16. Se for, o código imprime uma mensagem informando sobre essa compatibilidade.

**Configuração de treinamento:**

Load base model: O código carrega o modelo base especificado pelo model_name. Este modelo é um modelo pré-treinado de grande porte que serve como base para o treinamento personalizado.

Set training parameters: O código define os parâmetros de treinamento, incluindo o número de épocas de treinamento, o tamanho do lote, a taxa de aprendizado, o decaimento de peso, o otimizador, o agendador de taxa de aprendizado e o intervalo de salvamento de checkpoints.

**Configuração do treinamento supervisionado:**

Set supervised fine-tuning parameters: O código define os parâmetros específicos para o treinamento supervisionado, que é o tipo de treinamento utilizado neste caso. Esses parâmetros incluem a configuração LoRA para o modelo e o comprimento máximo da sequência.

**Treinamento do modelo:**

Trainer = SFTTrainer(model=model, train_dataset=dataset, peft_config=peft_config, dataset_text_field="text", max_seq_length=max_seq_length, tokenizer=tokenizer, args=training_arguments, packing=packing): Esta linha de código cria uma instância do objeto SFTTrainer, que é responsável por gerenciar o treinamento do modelo.

trainer.train(): Esta linha de código inicia o treinamento do modelo utilizando as configurações definidas anteriormente.

**Salvamento do modelo treinado:**

trainer.model.save_pretrained(new_model): Esta linha de código salva o modelo treinado no diretório especificado pelo new_model. Este modelo pode ser usado posteriormente para fazer previsões ou inferências.

In [ ]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")
#dataset = load_dataset(dataset_name, split="train", text_column="key", label_column="target")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    #dataset_text_field="text",
    dataset_text_field="prime",  # Use o nome correto da coluna de texto (entrada)
    #dataset_label_field="target",  # Use o nome correto da coluna de saída
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained("/content/sample_data/prime.ckpt")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/639976 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,8.298400
50,9.339900
75,8.042300
100,8.941400
125,7.605200
150,8.161400
175,6.658800
200,6.947600
225,5.504100
250,5.250900


In [ ]:
%load_ext tensorboard
%tensorboard --logdir results/runs

In [ ]:
dataset[1]

Ignora avisos: A linha logging.set_verbosity(logging.CRITICAL) ignora todos os avisos gerados durante a execução do código. Isso pode ser útil para suprimir mensagens desnecessárias e melhorar a legibilidade do código.

Executa pipeline de geração de texto com o modelo treinado: A linha pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200) inicializa um pipeline de geração de texto usando o modelo treinado (model) e o tokenizador (tokenizer). O pipeline é configurado para gerar até 200 tokens.

Fornece o prompt de entrada: A linha prompt = "Como puedo encontrar trabajo de ingeniero?" define o prompt de entrada para o pipeline, que é uma pergunta sobre como encontrar um emprego de engenheiro.

Processa e gera texto: A linha result = pipe(f"<s>[INST] {prompt} [/INST]") processa o prompt de entrada, aplicando a atenção LoRA (LoraAttention) e gerando uma resposta. O símbolo [INST] marca o início do prompt e [/INST] marca o fim do prompt.

Imprime a resposta generada: A linha print(result[0]['generated_text']) imprime a primeira resposta gerada pelo pipeline, que é uma sugestão sobre como encontrar um emprego de engenheiro.

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Como puedo encontrar trabajo de ingeniero?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"